Set-up

In [64]:
import requests
import pandas as pd
import numpy as np
import urllib.request

Function to calculate distance given latitude and longitude

In [65]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):

    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r *1000

Get LTA camera id and images

In [66]:
traffic_image_url='http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2'
headers_val={'AccountKey':'AO4qMbK3S7CWKSlplQZqlA=='}
traffic_image_req=requests.get(url=traffic_image_url,headers=headers_val)
traffic_image_df=pd.DataFrame(eval(traffic_image_req.content)['value'])
traffic_image_df['Count']=np.random.uniform(low=0, high=20, size=(len(traffic_image_df.index),)).astype(int)
traffic_image_df['is_jam']=0
traffic_image_df=traffic_image_df.merge(pd.read_csv('traffic_camera_region_RoadName.csv',converters={'CameraID':str}),'left','CameraID')

Get LTA incidents on Expressways

In [68]:
traffic_incidents_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents'
traffic_incidents_req=requests.get(url=traffic_incidents_url,headers=headers_val)
traffic_incidents_df=pd.DataFrame(eval(traffic_incidents_req.content)['value'])
incidents_roads=['AYE','BKE','CTE','ECP','KJE','KPE','MCE','PIE','SLE','TPE','Sentosa','Tuas','Woodlands']
traffic_incidents_df=traffic_incidents_df[traffic_incidents_df['Message'].apply(lambda x: any(expressway in x for expressway in incidents_roads))]

NEA API to get rainfall in mm

In [70]:
weatherreq=requests.get(url='https://api.data.gov.sg/v1/environment/rainfall')
weather_df=pd.DataFrame(eval(weatherreq.content)['metadata']['stations'])

weather_df['latitude']=weather_df['location'].apply(lambda x: x['latitude'])
weather_df['longitude']=weather_df['location'].apply(lambda x: x['longitude'])
weather_df['timestamp']=eval(weatherreq.content)['items'][0]['timestamp']
weather_df['timestamp']=pd.to_datetime(weather_df['timestamp'])

station_rainfall=pd.DataFrame(eval(weatherreq.content)['items'][0]['readings']).rename(columns={'value':'rainfall'})

weather_df=weather_df.merge(station_rainfall,how='left',left_on='id',right_on='station_id')
weather_df=weather_df.drop(['id','device_id','station_id','location'],axis=1)

Calculations and table joining


In [71]:
traffic_image_df['key']=0
traffic_incidents_df['key']=0
weather_df['key']=0

Select incidents that occur within 500m of a camera location

In [72]:
nearest_incidents=traffic_image_df.merge(traffic_incidents_df,'outer','key')
nearest_incidents['incident_distance_from_id']=(np.vectorize(haversine)(nearest_incidents['Latitude_x'],nearest_incidents['Longitude_x'],nearest_incidents['Latitude_y'],nearest_incidents['Longitude_y']))
nearest_incidents=nearest_incidents[nearest_incidents['incident_distance_from_id']<500].sort_values('incident_distance_from_id')
nearest_incidents=nearest_incidents[['CameraID','Message']]

Select nearest weather station for each camera id

In [73]:
final_df=traffic_image_df.merge(weather_df,'outer','key')
final_df['distance_from_id']=(np.vectorize(haversine)(final_df['Latitude'],final_df['Longitude'],final_df['latitude'],final_df['longitude']))
final_df=final_df.sort_values('distance_from_id').groupby('CameraID').head(1)[['CameraID','Latitude','Longitude','Region','rainfall','ImageLink','RoadName','Count','is_jam']]
final_df=final_df.sort_values('CameraID').reset_index(drop=True)


Convert dataframes to CSV to be used in frontend

In [74]:
final_df.to_csv('main_df.csv',index=False)
nearest_incidents.to_csv('traffic_incidents.csv',index=False)

# END

In [2]:
import pandas as pd

In [3]:
main_df=pd.read_csv('main_df.csv')
camera = []
for i in range(len(main_df)):
    d=dict(name = main_df.iloc[i,6], lat = main_df.iloc[i,1], lon = main_df.iloc[i,2])
    camera.append(d)
camera

[{'name': 'East Coast Parkway (ECP)', 'lat': 1.29531332, 'lon': 103.871146},
 {'name': 'Pan-Island Expressway (PIE)',
  'lat': 1.319541067,
  'lon': 103.8785627},
 {'name': 'Pan-Island Expressway (PIE)',
  'lat': 1.323957439,
  'lon': 103.8728576},
 {'name': 'Kallang Way Flyover', 'lat': 1.319535712, 'lon': 103.8750668},
 {'name': 'Kallang-Paya Lebar Expressway (KPE)',
  'lat': 1.363519886,
  'lon': 103.905394},
 {'name': 'Kallang-Paya Lebar Expressway (KPE)',
  'lat': 1.357098686,
  'lon': 103.902042},
 {'name': 'Marina Coastal Expressway (MCE)',
  'lat': 1.27414394350065,
  'lon': 103.851316802547},
 {'name': 'Marina Coastal Expressway (MCE)',
  'lat': 1.27135090682664,
  'lon': 103.861828440597},
 {'name': 'Marina Coastal Expressway (MCE)',
  'lat': 1.27066408655104,
  'lon': 103.856977943394},
 {'name': 'Marina Coastal Expressway (MCE)',
  'lat': 1.29409891409364,
  'lon': 103.876056196568},
 {'name': 'Marina Coastal Expressway (MCE)',
  'lat': 1.2752977149006,
  'lon': 103.8663903

In [10]:
# Create drop down options.
dd_options = [dict(value=c["name"], label=c["name"]+'_') for c in camera]
dd_defaults = [o["value"] for o in dd_options]

In [11]:
dd_options


[{'value': 'East Coast Parkway (ECP)', 'label': 'East Coast Parkway (ECP)lul'},
 {'value': 'Pan-Island Expressway (PIE)',
  'label': 'Pan-Island Expressway (PIE)lul'},
 {'value': 'Pan-Island Expressway (PIE)',
  'label': 'Pan-Island Expressway (PIE)lul'},
 {'value': 'Kallang Way Flyover', 'label': 'Kallang Way Flyoverlul'},
 {'value': 'Kallang-Paya Lebar Expressway (KPE)',
  'label': 'Kallang-Paya Lebar Expressway (KPE)lul'},
 {'value': 'Kallang-Paya Lebar Expressway (KPE)',
  'label': 'Kallang-Paya Lebar Expressway (KPE)lul'},
 {'value': 'Marina Coastal Expressway (MCE)',
  'label': 'Marina Coastal Expressway (MCE)lul'},
 {'value': 'Marina Coastal Expressway (MCE)',
  'label': 'Marina Coastal Expressway (MCE)lul'},
 {'value': 'Marina Coastal Expressway (MCE)',
  'label': 'Marina Coastal Expressway (MCE)lul'},
 {'value': 'Marina Coastal Expressway (MCE)',
  'label': 'Marina Coastal Expressway (MCE)lul'},
 {'value': 'Marina Coastal Expressway (MCE)',
  'label': 'Marina Coastal Expresswa